# Socio-economic Profiles 2019

This code creates the socio-economic profile data for the San Francisco Planning Department's Neighborhood Socio-Economic Profiles. Socio-economic profiles data is derived from the [American Community Survey](https://www.census.gov/programs-surveys/acs) 5-year data and is created annually by the Planning Department. Tract level socio-economic data from is combined at the neighborhood and zip code level for the City of San Francisco. This code is based off methods created by Michael Webster and others. Run the notebook to:

- Download ACS data using the Census API
- Calculate socio-economic profiles data
- Export data to csv in two formats. 


## Import packages

In [106]:
# base libraries
import requests, json
import pandas as pd
from collections import defaultdict

# graph libraries
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# map libraries
import folium
import branca.colormap as cm
from ipyleaflet import Map, GeoData, basemaps, LayersControl, Choropleth, linear, LegendControl

import geopandas

## Retrieve data from Census API
All socio-economic data comes from the Census ACS 5-year estimates and is available at the tract level through the census API. API documentation and data for the 2019 ACS data and previous years is available [here](https://www.census.gov/data/developers/data-sets/acs-5year.html)

### Census Attribute IDs
The attribute ID list below is a list of ASC attribute IDs that correspond to attributes included in the socio-economic data calcs. For a list of attribute ID and their meanings visit the API docs [here](https://api.census.gov/data/2019/acs/acs5/variables.html). 

Attributes included in the list below were derived from the attributes in the [Data_Items_and_Sources.xlsx](https://github.com/jsherba/socio-economic-profiles/raw/main/Data_Items_And_Sources_2019.xlsx) file developed by Michael Webster.

In [2]:
# list of attribute ids
attribute_lookup_df = pd.read_csv (r'./attribute_lookup.csv', dtype=str)
attribute_ids_extracted = attribute_lookup_df['attribute_id'].tolist()
attribute_ids = []
for attribute_id in attribute_ids_extracted:
    attribute_ids.extend(attribute_id.split(", "))
attribute_ids = list(set([x+"E" for x in attribute_ids]))
attribute_ids[:10]

['B15003_002E',
 'B11001_007E',
 'B25056_016E',
 'B19001_015E',
 'B25063_011E',
 'B25063_012E',
 'B25070_002E',
 'B25038_008E',
 'B25044_002E',
 'B25063_020E']

### Build Census API URL and Make Query
The code below builds the Url for the census API call to get relevant 2019 ACS attribute data at the tract level for San Francisco County.The Census API accepts up to 50 attributes at a time. Therefore a the attribute list is first grouped into sublists of 45. A recursive function queries the API for each group of 45 attributes and appends results together. Below define:
- Tract code is '*' to collect all tracts
- State code is '06' for CA
- County code is '075' for San Francisco County
- Attributes are defined by the attribute id list and includes all relevant attributes for the socio-economic data calcs

In [3]:
# function builds the api URL from tract_code, state_code, county_code, and attribute ids. 
def build_census_url(tract_code, state_code, county_code, attribute_ids):
    attributes = ','.join(attribute_ids)
    census_url = r'https://api.census.gov/data/2019/acs/acs5?get={}&for=tract:{}&in=state:{}&in=county:{}'\
                .format(attributes, tract_code, state_code, county_code)
    return census_url
    

In [4]:
# function makes a single api call and collects results in a pandas dataframe
def make_census_api_call(census_url):
    # make API call to Census
    resp = requests.get(census_url)
    if resp.status_code != 200:
        # this means something went wrong
        resp.raise_for_status()
       
    # retrieve data as json and convert to Pandas Dataframe
    data = resp.json()
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)

    # convert values that are not state, county, or tract to numeric type
    cols=[i for i in df.columns if i not in ["state","county","tract"]]
    for col in cols:
        df[col]=pd.to_numeric(df[col])
        
    return df

In [5]:
# set geo variables for api call
tract_code = "*"
state_code = "06"
county_code = "075"

# split attributes into groups of 45, run a census query for each, merge outputs into a single df
split_attribute_ids = [attribute_ids[i:i+45] for i in range(0, len(attribute_ids), 45)]
df=None
first = True
for ids in split_attribute_ids:
    census_url = build_census_url(tract_code, state_code, county_code, ids)
    returned_df = make_census_api_call(census_url)
    if first:
        df = returned_df
        first = False
    else:
        returned_df = returned_df.drop(columns=['state', 'county'])
        df = pd.merge(df, returned_df, on='tract', how='left')

df.head()

,B15003_002E,B11001_007E,B25056_016E,B19001_015E,B25063_011E,B25063_012E,B25070_002E,B25038_008E,B25044_002E,B25063_020E,...,B15003_019E,B17001_001E,B19001_005E,B07003_016E,B25038_006E,B25075_021E,B19101_016E,B25075_015E,B25044_009E,B19101_007E
0,8,200,0,103,0,0,25,185,671,12,...,29,2507,14,34,181,0,112,0,220,0
1,248,102,18,49,0,0,14,132,407,15,...,220,2877,38,69,143,17,27,0,308,37
2,0,250,7,62,0,0,0,198,749,14,...,47,2486,25,8,198,0,88,0,150,21
3,47,306,9,218,0,0,14,320,1086,11,...,70,3804,28,104,308,12,228,12,358,30
4,0,453,0,7,0,0,42,0,0,85,...,15,1838,66,60,0,0,23,0,645,9


## Prepare Lookup Dictionaries and Helper Functions

### Tract/Neighborhood Lookup
Two lookup dictionaries are created below that relate neghborhoods to tracts and supervisor districts to tracts. These dictionaries are used to subset the census dataframe for each neighborhood or district so that calcs can be run on each set of tracts. The lookup dictionaries are created from geo_lookup.csv in the repository.

In [6]:
# import geo_lookup csv
geo_lookup_df = pd.read_csv (r'./geo_lookup.csv', dtype=str)

tract_nb_lookup = defaultdict(list)
tract_sd_lookup = defaultdict(list)

# create tract lookup dictionary for neighborhoods
for i, j in zip(geo_lookup_df['neighborhood'], geo_lookup_df['tractid']):
    tract_nb_lookup[i].append(j)

# create tract lookup dictionary for supervisor districts
for i, j in zip(geo_lookup_df['supervisor_district'], geo_lookup_df['tractid']):
    tract_sd_lookup[i].append(j)


### Calculating Medians
To calculate median values of aggregated geographies you cannot use the mean of component geographies. Instead a statistical approximation of the median must be calculated from range tables. 

Range variables in the ACS have a unique ID like any other Census variable. They represent the amount of a variable within a select range. e.g. number of households with household incomes between $45000-50000. Range variable ID's and range information is stored in the median_ranges.csv file in the repository. These range variables and ranges are needed for calculating the median at the neighborhood level. 

The below function calculates a median based on range data. This method follows the offical ACS documentation for [calculating a median](https://www.dof.ca.gov/Forecasting/Demographics/Census_Data_Center_Network/documents/How_to_Recalculate_a_Median.pdf)


In [7]:
# import median tables from median_ranges csv and add empty columns for rows 'households and 'cumulative_totals'
range_df = pd.read_csv (r'./median_ranges.csv')
range_df['households']=0
range_df['cumulative_total']=0
range_df.head()

,name,id,range_start,range_end,households,cumulative_total
0,median_household_income,B19001_002E,2500.0,9999.0,0,0
1,median_household_income,B19001_003E,10000.0,14999.0,0,0
2,median_household_income,B19001_004E,15000.0,19999.0,0,0
3,median_household_income,B19001_005E,20000.0,24999.0,0,0
4,median_household_income,B19001_006E,25000.0,29999.0,0,0


In [8]:
# define median helper function
def calc_median(tract_df, range_df, median_to_calc):
    
    # subset range df for current median variable to calc
    range_df = range_df[range_df['name']==median_to_calc]
    
    # sort dataframe low to high by range start column
    range_df = range_df.sort_values(by=['range_start'])
    
    # calculate households as sum of tract level households for each row based on range id
    range_df['households'] = range_df.apply(lambda row : tract_df[row['id']].sum(), axis = 1)
    
    # calculate the cumulative total of households
    range_df['cumulative_total'] = range_df['households'].cumsum()
    
    # calculate total households and return 0 if total households is 0
    total_households = range_df['households'].sum()
    
    # if total households is 0 set median to 0
    if total_households == 0:
        return 0
    
    # calculate midpoint
    midpoint = total_households/2

    # if midpoint is below first range return median as end of first range value
    if midpoint < range_df['cumulative_total'].min():
        new_median = range_df['range_end'].min()
        return new_median
    
    # if midpoint is above last range set median to end of last range value
    if midpoint > range_df['cumulative_total'].max():
        new_median = range_df['range_end'].max()
        return new_median
    
    less_midpoint_df = range_df[range_df['cumulative_total']<midpoint]
    
    # get the single row containing the range just below the mid range by getting the row with the max range start from the subsetted median df
    range_below_mid_range_df = less_midpoint_df[less_midpoint_df['range_start'] == less_midpoint_df['range_start'].max()]
    
    # get the cumulative total value for the first row of the range below mid range dictionary
    total_hh_previous_range = range_below_mid_range_df['cumulative_total'].iloc[0]
    hh_to_mid_range = midpoint - total_hh_previous_range
    
    # extract rows above midrange by subsetting median df for rows with cumulative total grearter than midpoint.
    greater_midpoint_df = range_df[range_df['cumulative_total']>midpoint]
    
    # get the single row containing the mid range by getting the row with the min range start from the subsetted median df
    mid_range_df = greater_midpoint_df[greater_midpoint_df['range_start'] == greater_midpoint_df['range_start'].min()]
    
    # get the households value for the first row of the mid range dictionary
    hh_in_mid_range = mid_range_df['households'].iloc[0]
    
    # calculate proportion of number of households in the mid range that would be needed to get to the mid-point
    prop_of_hh = hh_to_mid_range/hh_in_mid_range
    
    # calculate width of the mid range
    width = (mid_range_df['range_end'].iloc[0]-mid_range_df['range_start'].iloc[0])+1
    
    # apply proportion to width of mid range
    prop_to_width = prop_of_hh*width
    beginning_of_mid_range = mid_range_df['range_start'].iloc[0]
    
    # calculate new median
    new_median = beginning_of_mid_range + prop_to_width
    
    return new_median

## Calculate Socio-economic Data
The `calc_socio_economic_data` function takes tract level data from the API call and the tract/neighborhood lookup dictionary. This function creates all of the socio-economic data calcs and returns a dictionary. The calcs in this function are derived from the [Data_Items_and_Sources.xlsx](https://github.com/jsherba/socio-economic-profiles/raw/main/Data_Items_And_Sources_2019.xlsx) file developed by Michael Webster

In [9]:
# define other helper functions
def calc_sum(df, attribute_id):
    return df[attribute_id].sum()

def calc_normalized(df, attribute_id, attribute_id2):
    if df[attribute_id2].sum() == 0:
        return 0
    else:
        return (df[attribute_id].sum()/df[attribute_id2].sum())

def calc_sum_normalized(df, attribute_list, attribute_id2):
    if df[attribute_id2].sum()==0:
        return 0
    else:
        sum_of_attributes = 0
        for attribute_id in attribute_list:
            sum_of_attributes+=df[attribute_id].sum()
        return sum_of_attributes/df[attribute_id2].sum()


In [10]:
# function runs all calcs for each neighborhood or supervisor district
def calc_socio_economic_data(df, tract_lookup):
    # create empty dictionary to add calculated attribute information to
    all_calc_data = defaultdict(dict) 
    # calculate all stats for each neighborhood
    for nb_name, tracts in tract_lookup.items():
        # extract attribute information for tracks associated with a neighborhood
        tract_df = df[df['tract'].isin(tracts)]
        # build dictionary with all stats for a neighborhood
        all_calc_data_nb = all_calc_data[nb_name]
        # population
        all_calc_data_nb["Total Population"] = calc_sum(tract_df, 'B01001_001E')
        all_calc_data_nb["Group Quarter Population"] = calc_sum(tract_df, 'B26001_001E')
        all_calc_data_nb["Percent Female"] = calc_normalized(tract_df, 'B01001_026E', 'B01001_001E')
        # household stats
        all_calc_data_nb["Housholds"] = calc_sum(tract_df, 'B11001_001E')
        all_calc_data_nb["Family Households"] = calc_normalized(tract_df, 'B11001_002E', 'B11001_001E')
        all_calc_data_nb["Non-Family Households"] = calc_normalized(tract_df, 'B11001_007E', 'B11001_001E')
        all_calc_data_nb["Single Person Households, % of Total"] = calc_normalized(tract_df, 'B11001_008E', 'B11001_001E')
        all_calc_data_nb["Households with Children, % of Total"] = calc_normalized(tract_df, 'B11005_002E', 'B11001_001E')
        all_calc_data_nb["Households with 60 years and older, % of Total"] = calc_normalized(tract_df, 'B11006_002E', 'B11001_001E')
        all_calc_data_nb["Average Household Size"] = calc_normalized(tract_df, 'B11002_001E', 'B11001_001E')
        all_calc_data_nb["Average Family Household Size"] = calc_normalized(tract_df, 'B11002_002E', 'B11001_002E')
        # race and ethnicity stats
        all_calc_data_nb["Asian"] = calc_normalized(tract_df, 'B02001_005E', 'B02001_001E')
        all_calc_data_nb["Black/African American"] = calc_normalized(tract_df, 'B02001_003E', 'B02001_001E')
        all_calc_data_nb["White"] = calc_normalized(tract_df, 'B02001_002E', 'B02001_001E')
        all_calc_data_nb["Native American Indian"] = calc_normalized(tract_df, 'B02001_005E', 'B02001_001E')
        all_calc_data_nb["Native Hawaiian/Pacific Islander"] = calc_normalized(tract_df, 'B02001_006E', 'B02001_001E')
        all_calc_data_nb["Other/Two or More Races"] = calc_sum_normalized(tract_df, ['B02001_008E', 'B02001_007E'], 'B02001_001E')
        all_calc_data_nb["% Latino (of Any Race)"] = calc_normalized(tract_df, 'B03001_003E', 'B03001_001E')
        # age
        all_calc_data_nb["0-4 Years"] = calc_sum_normalized(tract_df, ['B01001_003E', 'B01001_027E'], 'B01001_001E')
        all_calc_data_nb["5-17 Years"] = calc_sum_normalized(tract_df, ['B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_028E', 'B01001_029E', 'B01001_030E'],'B01001_001E')
        all_calc_data_nb["18-34 Years"] = calc_sum_normalized(tract_df, ['B01001_007E','B01001_008E','B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E','B01001_031E','B01001_032E','B01001_033E','B01001_034E','B01001_035E','B01001_036E'], 'B01001_001E')
        all_calc_data_nb["35-59 Years"] = calc_sum_normalized(tract_df, ['B01001_013E', 'B01001_014E', 'B01001_015E', 'B01001_016E', 'B01001_017E', 'B01001_037E', 'B01001_038E', 'B01001_039E', 'B01001_040E', 'B01001_041E'], 'B01001_001E')
        all_calc_data_nb["60 and Older"] = calc_sum_normalized(tract_df, ['B01001_018E', 'B01001_019E', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_042E', 'B01001_043E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E'], 'B01001_001E')
        #all_calc_data_nb["Median Age"]
        # educationa attainment
        all_calc_data_nb["High School or Less"] = calc_sum_normalized(tract_df, ['B15003_002E', 'B15003_003E', 'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E', 'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E', 'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E', 'B15003_018E'], 'B15003_001E')
        all_calc_data_nb["Some College/Associate Degree"] = calc_sum_normalized(tract_df, ['B15003_019E', 'B15003_020E', 'B15003_021E'], 'B15003_001E')
        all_calc_data_nb["College Degree"] = calc_normalized(tract_df, 'B15003_022E', 'B15003_001E')
        all_calc_data_nb["Graduate/Professional Degree"] = calc_sum_normalized(tract_df, ['B15003_023E', 'B15003_024E', 'B15003_025E'], 'B15003_001E')
        # nativity
        all_calc_data_nb["Foreign Born"] = calc_normalized(tract_df, 'B05002_013E', 'B05002_001E')
        # language spoken at home
        all_calc_data_nb["English Only"] = calc_sum_normalized(tract_df, ['B16007_003E', 'B16007_009E', 'B16007_015E'], 'B16007_001E')
        all_calc_data_nb["Spanish Only"] = calc_sum_normalized(tract_df, ['B16007_004E', 'B16007_010E', 'B16007_016E'], 'B16007_001E')
        all_calc_data_nb["Asian/Pacific Islander"] = calc_sum_normalized(tract_df, ['B16007_006E', 'B16007_012E', 'B16007_018E'], 'B16007_001E')
        all_calc_data_nb["Other European Languages"] = calc_sum_normalized(tract_df, ['B16007_005E', 'B16007_011E', 'B16007_017E'], 'B16007_001E')
        all_calc_data_nb["Other Languages"] = calc_sum_normalized(tract_df, ['B16007_007E', 'B16007_013E', 'B16007_019E'], 'B16007_001E')
        # linguistic isolation
        all_calc_data_nb["% of All Households"] = calc_sum_normalized(tract_df, ['B16003_002E', 'B16003_008E'], 'B16004_001E')
        all_calc_data_nb["% of Spanish-Speaking Households"] = calc_sum_normalized(tract_df, ['B16003_004E', 'B16003_009E'], 'B16004_001E')
        all_calc_data_nb["% of Asian-Speaking Households"] = calc_sum_normalized(tract_df, ['B16003_006E', 'B16003_011E'], 'B16004_001E')
        all_calc_data_nb["% of Other European-Speaking Households"] = calc_sum_normalized(tract_df, ['B16003_005E', 'B16003_010E'], 'B16004_001E')
        all_calc_data_nb["% of Households Speaking Other Languages"] = calc_sum_normalized(tract_df, ['B16003_007E', 'B16003_012E'], 'B16004_001E')
        # housing
        all_calc_data_nb["Total Number of Units"] = calc_sum(df, 'B25001_001E')
        all_calc_data_nb["Median Year Structure Built"] = calc_median(tract_df, range_df, 'median_year_structure_built')
        all_calc_data_nb["Owner Occupied"] = calc_normalized(tract_df, 'B25007_002E', 'B25007_001E')
        all_calc_data_nb["Renter Occupied"] = calc_normalized(tract_df, 'B25007_012E', 'B25007_001E')
        all_calc_data_nb["Vacant Units"] = calc_normalized(tract_df, 'B25004_001E', 'B25001_001E')
        all_calc_data_nb["For Rent"] = calc_normalized(tract_df, 'B25004_002E', 'B25004_001E')
        all_calc_data_nb["For Sale Only"] = calc_normalized(tract_df, 'B25004_004E', 'B25004_001E')
        all_calc_data_nb["Rented or Sold, Not Occupied"] = calc_sum_normalized(tract_df, ['B25004_003E', 'B25004_005E'], 'B25004_001E')
        all_calc_data_nb["For Seasonal, Recreation or Occasional Use"] = calc_normalized(tract_df, 'B25004_006E', 'B25004_001E')
        all_calc_data_nb["Other Vacant"] = calc_normalized(tract_df, 'B25004_008E', 'B25004_001E')
        all_calc_data_nb["Median Year Moved in to Unit (Own)"] = calc_median(tract_df, range_df, 'median_year_moved_owner')
        all_calc_data_nb["Median Year Moved in to Unit (Rent)"] = calc_median(tract_df, range_df, 'median_year_moved_renter')
        all_calc_data_nb["Percent in Same House Last Year"] = calc_normalized(tract_df, 'B07001_017E', 'B07001_001E')
        all_calc_data_nb["Percent Abroad Last Year"] = calc_normalized(tract_df, 'B07003_016E', 'B07003_001E')
        # structure type
        all_calc_data_nb["Single Family Housing"] = calc_sum_normalized(tract_df, ['B25024_002E', 'B25024_003E'], 'B25024_001E')
        all_calc_data_nb["2-4 Units"] = calc_sum_normalized(tract_df, ['B25024_004E', 'B25024_005E'], 'B25024_001E')
        all_calc_data_nb["5-9 Units"] = calc_normalized(tract_df, 'B25024_006E', 'B25024_001E')
        all_calc_data_nb["10-19 Units"] = calc_normalized(tract_df, 'B25024_007E', 'B25024_001E')
        all_calc_data_nb["20 Units or More"] = calc_sum_normalized(tract_df, ['B25024_008E', 'B25024_009E'], 'B25024_001E')
        all_calc_data_nb["Other"] = calc_sum_normalized(tract_df, ['B25024_010E', 'B25024_011E'], 'B25024_001E')
        # unit size
        all_calc_data_nb["No Bedroom"] = calc_normalized(tract_df,'B25041_002E', 'B25041_001E')
        all_calc_data_nb["1 Bedroom"] = calc_normalized(tract_df, 'B25041_003E', 'B25041_001E')
        all_calc_data_nb["2 Bedrooms"] = calc_normalized(tract_df, 'B25041_004E', 'B25041_001E')
        all_calc_data_nb["3-4 Bedrooms"] = calc_sum_normalized(tract_df, ['B25041_005E', 'B25041_006E'], 'B25041_001E')
        all_calc_data_nb["5 Bedrooms"] = calc_normalized(tract_df, 'B25041_007E', 'B25041_001E')
        # housing prices
        all_calc_data_nb["Median Rent"] = calc_median(tract_df, range_df, 'median_rent')
        all_calc_data_nb["Median Contract Rent"] = calc_median(tract_df, range_df, 'median_rent_contract')
        all_calc_data_nb["Median Rent as % of Household Income"] = calc_median(tract_df, range_df, 'median_rent_percent_of_income')
        all_calc_data_nb["Median Home Value"] = calc_median(tract_df, range_df, 'median_home_value')
        # vehicles available
        all_calc_data_nb["Vehicles Available"] = calc_sum(df, 'B25046_001E')
        all_calc_data_nb["Vehicles Homeowners"] = calc_normalized(tract_df, 'B25046_002E', 'B25046_001E')
        all_calc_data_nb["Vehicles Renters"] = calc_normalized(tract_df, 'B25046_003E', 'B25046_001E')
        all_calc_data_nb["Vehicles Per Capita"] = calc_normalized(tract_df, 'B25046_001E', 'B01001_001E')
        all_calc_data_nb["Households with no Vehicle"] = calc_sum_normalized(tract_df, ['B25044_003E', 'B25044_010E'], 'B25044_001E')
        all_calc_data_nb["Percent of Homeowning Households"] = calc_normalized(tract_df, 'B25044_003E', 'B25044_002E')
        all_calc_data_nb["Percent of Renting Households"] = calc_normalized(tract_df, 'B25044_010E', 'B25044_009E')
        # income
        all_calc_data_nb["Median Household Income"] = calc_median(tract_df, range_df, 'median_household_income')
        all_calc_data_nb["Median Family Income"] = calc_median(tract_df, range_df, 'median_family_income')
        all_calc_data_nb["Per Capita Income"] = calc_normalized(tract_df, 'B19025_001E', 'B01001_001E')
        all_calc_data_nb["Percent in Poverty"] = calc_normalized(tract_df, 'B17001_002E', 'B17001_001E')
        # employment
        all_calc_data_nb["Unemployment Rate"] = calc_normalized(tract_df, 'B23025_005E', 'B23025_002E')
        all_calc_data_nb["Percent Unemployment Female"] = calc_sum_normalized(tract_df, ['B23001_094E', 'B23001_101E', 'B23001_108E', 'B23001_115E', 'B23001_122E', 'B23001_129E', 'B23001_136E', 'B23001_143E', 'B23001_150E', 'B23001_157E', 'B23001_162E', 'B23001_167E', 'B23001_172E', 'B23001_090E', 'B23001_097E', 'B23001_104E', 'B23001_111E', 'B23001_118E', 'B23001_125E', 'B23001_132E', 'B23001_139E', 'B23001_146E', 'B23001_153E', 'B23001_160E', 'B23001_165E', 'B23001_170E'], 'B23025_002E')
        all_calc_data_nb["Percent Unemployment Male"] = calc_sum_normalized(tract_df, ['B23001_008E', 'B23001_015E', 'B23001_022E', 'B23001_029E', 'B23001_036E', 'B23001_043E', 'B23001_050E', 'B23001_057E', 'B23001_064E', 'B23001_071E', 'B23001_076E', 'B23001_081E', 'B23001_086E', 'B23001_004E', 'B23001_011E', 'B23001_018E', 'B23001_025E', 'B23001_032E', 'B23001_039E', 'B23001_046E', 'B23001_053E', 'B23001_060E', 'B23001_067E', 'B23001_074E', 'B23001_079E', 'B23001_084E'], 'B23025_002E')
        all_calc_data_nb["Employed Residents"] = calc_sum(df, 'C24050_001E')
        all_calc_data_nb["Managerial Professional"] = calc_normalized(tract_df, 'C24050_015E', 'C24050_001E')
        all_calc_data_nb["Services"] = calc_normalized(tract_df, 'C24050_029E', 'C24050_001E')
        all_calc_data_nb["Sales and Office"] = calc_normalized(tract_df, 'C24050_043E', 'C24050_001E')
        all_calc_data_nb["Natural Resources"] = calc_normalized(tract_df, 'C24050_057E', 'C24050_001E')
        all_calc_data_nb["Production Transport Materials"] = calc_normalized(tract_df, 'C24050_071E', 'C24050_001E')
        # journey to work
        all_calc_data_nb["Workers 16 Years and Older"] = calc_sum(df, 'B08006_001E')
        all_calc_data_nb["Car"] = calc_normalized(tract_df, 'B08006_002E', 'B08006_001E')
        all_calc_data_nb["Drove Alone"] = calc_normalized(tract_df, 'B08006_003E', 'B08006_001E')
        all_calc_data_nb["Carpooled"] = calc_normalized(tract_df, 'B08006_004E', 'B08006_001E')
        all_calc_data_nb["Transit"] = calc_normalized(tract_df, 'B08006_008E', 'B08006_001E')
        all_calc_data_nb["Bike"] = calc_normalized(tract_df, 'B08006_014E', 'B08006_001E')
        all_calc_data_nb["Walk"] = calc_normalized(tract_df, 'B08006_015E', 'B08006_001E')
        all_calc_data_nb["Other"] = calc_normalized(tract_df, 'B08006_016E', 'B08006_001E')
        all_calc_data_nb["Worked at Home"] = calc_normalized(tract_df, 'B08006_017E', 'B08006_001E')
        # population density
        all_calc_data_nb["Population Density per Acre"] = calc_sum(df, 'B01001_001E')
            
    #return calc dictionary
    return all_calc_data

In [11]:
# run functions to calculate all stats and convert calc dictionary to pandas dataframe
all_calc_data = calc_socio_economic_data(df, tract_nb_lookup)
df_all_calcs = pd.DataFrame.from_dict(all_calc_data).reset_index()
df_all_calcs.rename(columns = {'index':'Attribute'}, inplace = True) 
df_all_calcs.head()


,Attribute,North Beach,Russian Hill,Financial District,Chinatown,Nob Hill,Tenderloin,Marina,Pacific Heights,Presidio Heights,...,Sunset/Parkside,Lakeshore,Inner Richmond,Outer Richmond,Seacliff,Presidio,Mission Bay,Lincoln Park,Golden Gate Park,McLaren Park
0,Total Population,11854.000000,18139.000000,21537.000000,14438.000000,26445.000000,29927.000000,25375.000000,24279.000000,10585.000000,...,82682.000000,14801.000000,22572.000000,45921.000000,2507.000000,4226.000000,13222.000000,312.000000,63.000000,507.000000
1,Group Quarter Population,17.000000,5.000000,563.000000,53.000000,977.000000,1523.000000,97.000000,518.000000,196.000000,...,452.000000,2723.000000,147.000000,202.000000,0.000000,0.000000,342.000000,191.000000,0.000000,0.000000
2,Percent Female,0.486081,0.503501,0.465617,0.516207,0.490641,0.429913,0.515153,0.518226,0.514218,...,0.507922,0.538950,0.547891,0.512336,0.483845,0.539991,0.519286,0.227564,0.396825,0.585799
3,Housholds,6297.000000,9636.000000,11297.000000,6907.000000,15247.000000,17649.000000,13583.000000,12815.000000,4805.000000,...,28277.000000,4924.000000,9385.000000,18652.000000,891.000000,1324.000000,6312.000000,70.000000,48.000000,239.000000
4,Family Households,0.397332,0.365816,0.372754,0.522948,0.268840,0.234348,0.309578,0.353726,0.470760,...,0.658839,0.390739,0.483324,0.537905,0.775533,0.556647,0.504119,0.442857,0.000000,0.472803


In [12]:
# transpose dataset for second neighborhood view of dataset
df_all_calcs2 = df_all_calcs.T.reset_index()
df_all_calcs2.columns = df_all_calcs2.iloc[0]
df_all_calcs2 = df_all_calcs2[1:].rename(columns={'Attribute':'Neighborhood'})
df_all_calcs2.head()

,Neighborhood,Total Population,Group Quarter Population,Percent Female,Housholds,Family Households,Non-Family Households,"Single Person Households, % of Total","Households with Children, % of Total","Households with 60 years and older, % of Total",...,Production Transport Materials,Workers 16 Years and Older,Car,Drove Alone,Carpooled,Transit,Bike,Walk,Worked at Home,Population Density per Acre
1,North Beach,11854.0,17.0,0.486081,6297.0,0.397332,0.602668,0.464983,0.112911,0.365094,...,0.043308,512455.0,0.227139,0.206834,0.020305,0.235821,0.045232,0.393082,0.071419,874961.0
2,Russian Hill,18139.0,5.0,0.503501,9636.0,0.365816,0.634184,0.408053,0.078456,0.354089,...,0.022665,512455.0,0.246709,0.196511,0.050198,0.346075,0.036479,0.218557,0.076606,874961.0
3,Financial District,21537.0,563.0,0.465617,11297.0,0.372754,0.627246,0.445694,0.102151,0.218642,...,0.032076,512455.0,0.200172,0.184107,0.016065,0.223625,0.023883,0.440221,0.092376,874961.0
4,Chinatown,14438.0,53.0,0.516207,6907.0,0.522948,0.477052,0.411901,0.159693,0.581439,...,0.089981,512455.0,0.153299,0.124243,0.029056,0.334292,0.010139,0.440527,0.039649,874961.0
5,Nob Hill,26445.0,977.0,0.490641,15247.0,0.26884,0.73116,0.556765,0.061127,0.273693,...,0.040428,512455.0,0.176066,0.139077,0.036989,0.275309,0.025095,0.407443,0.05867,874961.0


## Visualizations

In [13]:
def f(Variable):
    fig = px.bar(df_all_calcs2, x='Neighborhood', y=Variable)
    fig.update_xaxes(tickangle=45)
    fig.show()

variables = df_all_calcs2.columns[1:]

interact(f, Variable=variables)

interactive(children=(Dropdown(description='Variable', options=('Total Population', 'Group Quarter Population'…

<function __main__.f(Variable)>

In [14]:
melted = pd.melt(df_all_calcs2, id_vars='Neighborhood', value_vars=['Renter Occupied', 'Owner Occupied'])

fig2 = px.bar(melted, x="Neighborhood", y="value", color=0, title="Renter vs Owner Occupied", labels={
                     "value": "Percent Occupied",
                     '0': ""
                 },)

fig2.update_xaxes(tickangle=45)
fig2.show()

In [98]:
print(linear.YlOrRd_04.index)
print(linear.YlOrRd_04.colors)

[0.0, 0.3333333333333333, 0.6666666666666666, 1.0]
[(1.0, 1.0, 0.6980392156862745, 1.0), (0.996078431372549, 0.8, 0.3607843137254902, 1.0), (0.9921568627450981, 0.5529411764705883, 0.23529411764705882, 1.0), (0.8901960784313725, 0.10196078431372549, 0.10980392156862745, 1.0)]


In [112]:
neighborhoods_df = geopandas.read_file(r"C:\Users\Jason Sherba\Documents\2021_03_10_socioeconomic_profiles\socio-economic-profiles\neighborhoods\neighborhoods\neighborhoods.shp")
neighborhoods_df = neighborhoods_df.to_crs("EPSG:4326")
df_all_calcs2 = df_all_calcs2[[ "Neighborhood", "Total Population"]]
nilpop=neighborhoods_df.merge(df_all_calcs2,left_on="nhood", right_on="Neighborhood")


mymap = folium.Map(location=[37.77,-122.44], zoom_start=11,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)

myscale = (nilpop["Total Population"].quantile((0,0.1,0.75,0.9,0.98,1))).tolist()
print(myscale)
mymap.choropleth(
 geo_data=nilpop,
 name='Choropleth',
 data=nilpop,
 columns=[ "Neighborhood", "Total Population"],
 key_on="feature.properties.Neighborhood",
 fill_color='YlGnBu',
 threshold_scale=myscale,
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='Resident foreign population in %',
 smooth_factor=0
)



mymap

[63.0, 3008.0, 25375.0, 38662.0, 63552.40000000007, 82682.0]


## Export

In [15]:
# export both dataset views to csv
df_all_calcs.to_csv(r'C:\Users\Jason Sherba\Downloads\data\test.csv', index = False)
df_all_calcs2.to_csv(r'C:\Users\Jason Sherba\Downloads\data\test2.csv', index = False)